<h2 align='center'>Codebasics ML Course: ML Flow Tutorial</h2>

In [7]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [8]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [10]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_smt, y_train_smt = smt.fit_resample(X_train, y_train)
np.unique(y_train_smt, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

### Track Experiments

In [12]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_smt, y_train_smt),
        (X_test, y_test)
    )
]

In [13]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [14]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [16]:
# Initialize MLflow
mlflow.set_experiment("Anomalys")
mlflow.set_tracking_uri("http://127.0.0.1:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/12/18 22:27:35 INFO mlflow.tracking.fluent: Experiment with name 'Anomalys' does not exist. Creating a new experiment.
2025/12/18 22:27:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/18 22:27:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/4/runs/49abd0606cd54c0c8e85bbd2751207a9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4


2025/12/18 22:27:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/4/runs/c34c54f7715541a5be541a374da44b12
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4


2025/12/18 22:27:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/4/runs/0418b058116445edb6818cc21f5cfd81
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4
🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/4/runs/e4450974dc9346b2a57f6bcebc4e1a4f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4


### Register the Model

In [17]:
import mlflow
print(mlflow.get_tracking_uri())

http://127.0.0.1:5000


In [18]:
run_id = "e4450974dc9346b2a57f6bcebc4e1a4f"
model_uri = f'runs:/{run_id}/model'  # ✅ Changed from "model_name" to "model"

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name="XGBClassifier_SMOTE")

Successfully registered model 'XGBClassifier_SMOTE'.
2025/12/18 22:29:15 WARNING mlflow.tracking._model_registry.fluent: Run with id e4450974dc9346b2a57f6bcebc4e1a4f has no artifacts at artifact path 'model', registering model based on models:/m-2de10170316243fc960fc86ecc685d7e instead
2025/12/18 22:29:15 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBClassifier_SMOTE, version 1


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/4/runs/e4450974dc9346b2a57f6bcebc4e1a4f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4


Created version '1' of model 'XGBClassifier_SMOTE'.


### Load the Model

In [22]:
model_name = "XGBClassifier_SMOTE"  # Underscores, not spaces
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

### Transition the Model to Production

In [24]:
model_name = "XGBClassifier_SMOTE"  # Correct name with underscores
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGBClassifier_SMOTE' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1766123149717, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1766123149717, metrics=None, model_id=None, name='anomaly-detection-prod', params=None, run_id='e4450974dc9346b2a57f6bcebc4e1a4f', run_link='', source='models:/XGBClassifier_SMOTE/1', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [43]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

Please refer to following to learn more about model registry

https://mlflow.org/docs/latest/model-registry.html#model-registry-workflows to learn 